In [3]:
import os

# --- PATH CONFIGURATION ---
deploy_dir = "/Users/chinmayeenanda/Desktop/Donaldson_Project/donaldson_deploy"

# 1. CRITICAL FIX: Force create the directory structure first
if not os.path.exists(deploy_dir):
    os.makedirs(deploy_dir, exist_ok=True)
    print(f"✅ Created directory: {deploy_dir}")
else:
    print(f"ℹ️ Directory already exists: {deploy_dir}")

# --- FILE CONTENT DEFINITIONS ---

code_main = """import io
import os
from fastapi import FastAPI, File, UploadFile, HTTPException, Security, status
from fastapi.security import APIKeyHeader
from PIL import Image
from ultralytics import YOLO

API_KEY = os.getenv("API_KEY", "donaldson-secret-key-123") 
API_KEY_NAME = "X-API-Key"
api_key_header = APIKeyHeader(name=API_KEY_NAME, auto_error=False)

async def get_api_key(api_key_header: str = Security(api_key_header)):
    if api_key_header == API_KEY:
        return api_key_header
    raise HTTPException(status_code=403, detail="Invalid API Key")

app = FastAPI(title="Donaldson Filter Inspector")

print("🚀 Loading YOLO Model...")
model = YOLO("best.pt") 

@app.post("/predict")
async def predict(file: UploadFile = File(...), api_key: str = Security(get_api_key)):
    try:
        image_data = await file.read()
        image = Image.open(io.BytesIO(image_data))
    except:
        raise HTTPException(status_code=400, detail="Invalid image")

    results = model.predict(image, conf=0.25)
    
    detections = []
    decision = "PASS"
    for box in results[0].boxes:
        decision = "REJECT" 
        name = results[0].names[int(box.cls[0])]
        detections.append({"type": name, "confidence": float(box.conf[0])})

    return {"decision": decision, "detections": detections}
"""

code_reqs = """fastapi
uvicorn
python-multipart
ultralytics
pillow
torch --index-url https://download.pytorch.org/whl/cpu
torchvision --index-url https://download.pytorch.org/whl/cpu
"""

code_docker = """FROM python:3.9-slim
RUN apt-get update && apt-get install -y libgl1-mesa-glx libglib2.0-0 && rm -rf /var/lib/apt/lists/*
WORKDIR /app
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt
COPY main.py .
COPY best.pt .
EXPOSE 80
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "80"]
"""

# --- WRITE FILES ---
files = {"main.py": code_main, "requirements.txt": code_reqs, "Dockerfile": code_docker}

for filename, content in files.items():
    with open(os.path.join(deploy_dir, filename), "w") as f:
        f.write(content)
    print(f"✅ Created: {filename}")

print(f"\n📂 Files are ready in: {deploy_dir}")
print("⚠️ NEXT STEP: Manually copy your 'best.pt' file into this folder!")

✅ Created directory: /Users/chinmayeenanda/Desktop/Donaldson_Project/donaldson_deploy
✅ Created: main.py
✅ Created: requirements.txt
✅ Created: Dockerfile

📂 Files are ready in: /Users/chinmayeenanda/Desktop/Donaldson_Project/donaldson_deploy
⚠️ NEXT STEP: Manually copy your 'best.pt' file into this folder!
